# 1. Cleans up region_distribution, potential_reach, demographic_distribution, spend, impressions columns. 
# 2. only keeping data where the ad_creation_time is between 8/1/2020 and 11/3/2020.
# 3. replacing all missing values with 0.
***

In [1]:
#loading the csv file.
import numpy as np
import pandas as pd
from collections import defaultdict

df = pd.read_csv("U.S. Senator Tina Smith_Senate.csv", delimiter = ',')

#df.dropna(axis=0,how='any',thresh=None,subset=None,inplace=True)
#df = df.reset_index(drop=True)
#df

****

### cleaning up region_distribution column

In [2]:
#breaking up nested dictionaries and keeping percentages into one dictionary and region into other.
result = defaultdict(list)
for item in range(0, len(df.region_distribution)):
    dfRegion = eval(df.region_distribution[item].replace('"', ""))
    for sub in dfRegion:
        for key in sub:
            result[key].append(sub[key])

In [3]:
#converting above dictionaries into dataframes.
new_df = pd.DataFrame(result)
print(new_df.shape)
new_df.head(600)

(16047, 2)


,percentage,region
0,1,Minnesota
1,1,Minnesota
2,1,Minnesota
3,1,Minnesota
4,1,Minnesota
...,...,...
595,0.002014,Idaho
596,0.006042,Iowa
597,0.001007,Montana
598,0.001007,Kansas


In [4]:
#one-hot encoding using .get_dummies library with respect to the region from above dataframe.
y = pd.get_dummies(new_df.region)
print(y.columns)

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Unknown', 'Utah', 'Vermont', 'Virginia',
       'Washington', 'Washington, District of Columbia', 'West Virginia',
       'Wisconsin', 'Wyoming'],
      dtype='object')


In [5]:
#storing states in a list.
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington","West Virginia", "Wisconsin", "Wyoming"]
print(len(state_names))

50


In [6]:
#dropping columns which are not states
for ele in y.columns:
    if ele not in state_names:
        y = y.drop(columns = [ele])
y.columns

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object')

In [7]:
#replacing 1s with actual percentage values.
y.replace(to_replace=[1], value = [new_df.percentage], inplace = True)

In [8]:
index = len(state_names) - len(y.columns)
new_states = []

#storing the states which are not in y in a list called new_states.
for ele in state_names:
    if ele not in y.columns:
        new_states.append(ele + "_region_distribution")

#adding _region_distribution at the end of state names in y.
for items in y.columns:
    y = y.rename(columns = {str(items) : str(items) + '_region_distribution'}, inplace = False)

#creating new columns in dataframe y with the states in new_states list.
for i in range(0, index):
    y[new_states[i]] = 0
y = y.reindex(sorted(y.columns), axis=1)
y.shape

(16047, 50)

In [9]:
"""dropping old region_distribution column first and then concating old (with dropped region_distribution column) 
and new dataframe y.
"""
df.drop('region_distribution', axis = 1, inplace=True)
output = pd.concat([y, df], axis = 1)

In [10]:
output.columns

Index(['Alabama_region_distribution', 'Alaska_region_distribution',
       'Arizona_region_distribution', 'Arkansas_region_distribution',
       'California_region_distribution', 'Colorado_region_distribution',
       'Connecticut_region_distribution', 'Delaware_region_distribution',
       'Florida_region_distribution', 'Georgia_region_distribution',
       'Hawaii_region_distribution', 'Idaho_region_distribution',
       'Illinois_region_distribution', 'Indiana_region_distribution',
       'Iowa_region_distribution', 'Kansas_region_distribution',
       'Kentucky_region_distribution', 'Louisiana_region_distribution',
       'Maine_region_distribution', 'Maryland_region_distribution',
       'Massachusetts_region_distribution', 'Michigan_region_distribution',
       'Minnesota_region_distribution', 'Mississippi_region_distribution',
       'Missouri_region_distribution', 'Montana_region_distribution',
       'Nebraska_region_distribution', 'Nevada_region_distribution',
       'New Ham

***

In [11]:
# Prepping to break publisher_platforms into two columns by adding two new columns facebook and instagram.
df['facebook'] = 0
df['instagram'] = 0
#df

In [12]:
#adding 1s in facebook if publisher_platforms has facebook and 1s in instagram if publisher_platforms has instagram.
for item in df.index:
    if 'facebook' in df['publisher_platforms'][item] and 'instagram' not in df['publisher_platforms'][item]:
        df['facebook'][item] = 1
    elif 'facebook' not in df['publisher_platforms'][item] and 'instagram' in df['publisher_platforms'][item]:
        df['instagram'][item] = 1
    elif 'facebook' in df['publisher_platforms'][item] and 'instagram' in df['publisher_platforms'][item]:
        df['facebook'][item] = 1
        df['instagram'][item] = 1

<ipython-input-12-5b0ae7752542>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['facebook'][item] = 1
<ipython-input-12-5b0ae7752542>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['instagram'][item] = 1
<ipython-input-12-5b0ae7752542>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['instagram'][item] = 1
<ipython-input-12-5b0ae7752542>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [13]:
df.loc[1:20, ['publisher_platforms', 'facebook', 'instagram']]
df1 = df.drop(columns = ['publisher_platforms','potential_reach','demographic_distribution','currency','id','ad_creation_time','ad_creative_body','ad_creative_link_caption','ad_creative_link_description','ad_creative_link_title','page_id','page_name','ad_delivery_start_time','ad_delivery_stop_time','funding_entity','spend','impressions','ad_snapshot_url'])

In [14]:
output.drop('publisher_platforms', axis = 1, inplace = True)
output1 = pd.concat([output, df1], axis = 1)

In [15]:
output1

,Alabama_region_distribution,Alaska_region_distribution,Arizona_region_distribution,Arkansas_region_distribution,California_region_distribution,Colorado_region_distribution,Connecticut_region_distribution,Delaware_region_distribution,Florida_region_distribution,Georgia_region_distribution,...,page_id,page_name,ad_delivery_start_time,ad_delivery_stop_time,funding_entity,spend,impressions,ad_snapshot_url,facebook,instagram
0,0,0,0,0,0,0,0,0,0,0,...,2.040270e+15,Tina Smith,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '600', 'upper_bound': '699'}","{'lower_bound': '30000', 'upper_bound': '34999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,2.040270e+15,Tina Smith,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '200', 'upper_bound': '299'}","{'lower_bound': '10000', 'upper_bound': '14999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0
2,0,0,0,0,0,0,0,0,0,0,...,2.040270e+15,Tina Smith,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '300', 'upper_bound': '399'}","{'lower_bound': '35000', 'upper_bound': '39999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0
3,0,0,0,0,0,0,0,0,0,0,...,2.040270e+15,Tina Smith,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '100', 'upper_bound': '199'}","{'lower_bound': '10000', 'upper_bound': '14999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0
4,0,0,0,0,0,0,0,0,0,0,...,2.040270e+15,Tina Smith,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '0', 'upper_bound': '99'}","{'lower_bound': '1000', 'upper_bound': '1999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16042,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16043,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16044,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16045,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***

In [16]:
df["potential_reach"] = df["potential_reach"].apply(lambda x : dict(eval(x)) )
df2 = df["potential_reach"].apply(pd.Series)

In [17]:
df2

,lower_bound,upper_bound
0,100001,500000
1,50001,100000
2,100001,500000
3,50001,100000
4,10001,50000
...,...,...
1381,10001,50000
1382,10001,50000
1383,5001,10000
1384,1000001,NaN


In [18]:
output1.drop('potential_reach', axis = 1, inplace=True)
output2 = pd.concat([output1, df2], axis = 1)
df3 = output2.rename(columns = {'lower_bound': 'lower_bound_potential_reach', 'upper_bound': 'upper_bound_potential_reach'}, inplace = False)
df3

,Alabama_region_distribution,Alaska_region_distribution,Arizona_region_distribution,Arkansas_region_distribution,California_region_distribution,Colorado_region_distribution,Connecticut_region_distribution,Delaware_region_distribution,Florida_region_distribution,Georgia_region_distribution,...,ad_delivery_start_time,ad_delivery_stop_time,funding_entity,spend,impressions,ad_snapshot_url,facebook,instagram,lower_bound_potential_reach,upper_bound_potential_reach
0,0,0,0,0,0,0,0,0,0,0,...,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '600', 'upper_bound': '699'}","{'lower_bound': '30000', 'upper_bound': '34999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0,100001,500000
1,0,0,0,0,0,0,0,0,0,0,...,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '200', 'upper_bound': '299'}","{'lower_bound': '10000', 'upper_bound': '14999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0,50001,100000
2,0,0,0,0,0,0,0,0,0,0,...,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '300', 'upper_bound': '399'}","{'lower_bound': '35000', 'upper_bound': '39999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0,100001,500000
3,0,0,0,0,0,0,0,0,0,0,...,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '100', 'upper_bound': '199'}","{'lower_bound': '10000', 'upper_bound': '14999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0,50001,100000
4,0,0,0,0,0,0,0,0,0,0,...,2020-10-24,2020-11-03,TINA SMITH FOR MINNESOTA,"{'lower_bound': '0', 'upper_bound': '99'}","{'lower_bound': '1000', 'upper_bound': '1999'}",https://www.facebook.com/ads/archive/render_ad...,1.0,1.0,10001,50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16042,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16043,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16044,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16045,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***

In [19]:
df["spend"] = df["spend"].apply(lambda x : dict(eval(x)))
df4 = df["spend"].apply(pd.Series)
#df4

In [20]:
df3.drop('spend', axis = 1, inplace=True)
output = pd.concat([df3, df4], axis = 1)
df5 = output.rename(columns = {'lower_bound': 'lower_bound_spend', 'upper_bound': 'upper_bound_spend'}, inplace = False)
#df5

***

In [21]:
df["impressions"] = df["impressions"].apply(lambda x : dict(eval(x)))
df6 = df["impressions"].apply(pd.Series)
#df6

In [22]:
df5.drop('impressions', axis = 1, inplace=True)
output = pd.concat([df5, df6], axis = 1)
df7 = output.rename(columns = {'lower_bound': 'lower_bound_impressions', 'upper_bound': 'upper_bound_impressions'}, inplace = False)
#df7

*** 

In [23]:
result = defaultdict(list)
for item in range(0, len(df.demographic_distribution)):
    dfRegion = eval(df.demographic_distribution[item].replace('"', ""))
    for sub in dfRegion:
        for key in sub:
            result[key].append(sub[key])

In [24]:
new_df = pd.DataFrame(result)
print(new_df.shape)
new_df.head(600)

(16727, 3)


,percentage,age,gender
0,0.058573,55-64,female
1,0.045846,65+,female
2,0.003758,25-34,unknown
3,0.000802,65+,unknown
4,0.001653,35-44,unknown
...,...,...,...
595,0.164252,65+,male
596,0.034127,35-44,male
597,0.066781,55-64,male
598,0.08495,45-54,female


In [25]:
distinct_age = pd.unique(new_df['age'])
print(distinct_age)
distinct_gender = pd.unique(new_df['gender'])
print(distinct_gender)

['55-64' '65+' '25-34' '35-44' '18-24' '45-54']
['female' 'unknown' 'male']


In [26]:
val1=[]
val2=[]
val3=[]
for item in new_df.index:
    if '35-44' in new_df['age'][item] and 'unknown' in new_df['gender'][item]:
        val1.append(new_df['percentage'][item])
    elif '35-44' in new_df['age'][item] and 'female' in new_df['gender'][item]:
        val2.append(new_df['percentage'][item])
    elif '35-44' in new_df['age'][item] and 'male' in new_df['gender'][item]:
        val3.append(new_df['percentage'][item])

unknown = pd.DataFrame({'35-44_unknown':val1})
female = pd.DataFrame({'35-44_female':val2})
male = pd.DataFrame({'35-44_male':val2})

In [27]:
out = pd.concat([unknown, female, male], axis = 1)
out

,35-44_unknown,35-44_female,35-44_male
0,0.001653,0.108628,0.108628
1,0.002191,0.197589,0.197589
2,0.001645,0.106398,0.106398
3,0.003457,0.278272,0.278272
4,0.002714,0.101764,0.101764
...,...,...,...
1184,NaN,0.051515,0.051515
1185,NaN,0.027027,0.027027
1186,NaN,0.078679,0.078679
1187,NaN,0.025735,0.025735


In [28]:
val4=[]
val5=[]
val6=[]
for item in new_df.index:
    if '65+' in new_df['age'][item] and 'unknown' in new_df['gender'][item]:
        val4.append(new_df['percentage'][item])
    elif '65+' in new_df['age'][item] and 'female' in new_df['gender'][item]:
        val5.append(new_df['percentage'][item])
    elif '65+' in new_df['age'][item] and 'male' in new_df['gender'][item]:
        val6.append(new_df['percentage'][item])

unknown = pd.DataFrame({'65+_unknown':val4})
female = pd.DataFrame({'65+_female':val5})
male = pd.DataFrame({'65+_male':val6})

In [29]:
out1 = pd.concat([out, unknown, female, male], axis = 1)
out1

,35-44_unknown,35-44_female,35-44_male,65+_unknown,65+_female,65+_male
0,0.001653,0.108628,0.108628,0.000802,0.045846,0.024702
1,0.002191,0.197589,0.197589,0.000974,0.099343,0.041028
2,0.001645,0.106398,0.106398,0.000866,0.03056,0.017834
3,0.003457,0.278272,0.278272,0.001481,0.084938,0.033333
4,0.002714,0.101764,0.101764,0.027027,0.014247,0.025102
...,...,...,...,...,...,...
1215,NaN,NaN,NaN,NaN,0.263636,NaN
1216,NaN,NaN,NaN,NaN,0.261261,NaN
1217,NaN,NaN,NaN,NaN,0.281321,NaN
1218,NaN,NaN,NaN,NaN,0.156894,NaN


In [30]:
val7=[]
val8=[]
val9=[]
for item in new_df.index:
    if '18-24' in new_df['age'][item] and 'unknown' in new_df['gender'][item]:
        val7.append(new_df['percentage'][item])
    elif '18-24' in new_df['age'][item] and 'female' in new_df['gender'][item]:
        val8.append(new_df['percentage'][item])
    elif '18-24' in new_df['age'][item] and 'male' in new_df['gender'][item]:
        val9.append(new_df['percentage'][item])

unknown = pd.DataFrame({'18-24_unknown':val7})
female = pd.DataFrame({'18-24_female':val8})
male = pd.DataFrame({'18-24_male':val9})

In [31]:
out2 = pd.concat([out1, unknown, female, male], axis = 1)
out2

,35-44_unknown,35-44_female,35-44_male,65+_unknown,65+_female,65+_male,18-24_unknown,18-24_female,18-24_male
0,0.001653,0.108628,0.108628,0.000802,0.045846,0.024702,0.001453,0.064435,0.052009
1,0.002191,0.197589,0.197589,0.000974,0.099343,0.041028,0.003117,0.005722,0.006696
2,0.001645,0.106398,0.106398,0.000866,0.03056,0.017834,0.000678,0.101983,0.05558
3,0.003457,0.278272,0.278272,0.001481,0.084938,0.033333,0.00041,0.006173,0.007407
4,0.002714,0.101764,0.101764,0.027027,0.014247,0.025102,0.000137,0.06038,0.101764
...,...,...,...,...,...,...,...,...,...
1215,NaN,NaN,NaN,NaN,0.263636,NaN,NaN,NaN,NaN
1216,NaN,NaN,NaN,NaN,0.261261,NaN,NaN,NaN,NaN
1217,NaN,NaN,NaN,NaN,0.281321,NaN,NaN,NaN,NaN
1218,NaN,NaN,NaN,NaN,0.156894,NaN,NaN,NaN,NaN


In [32]:
val10=[]
val11=[]
val12=[]
for item in new_df.index:
    if '25-34' in new_df['age'][item] and 'unknown' in new_df['gender'][item]:
        val10.append(new_df['percentage'][item])
    elif '25-34' in new_df['age'][item] and 'female' in new_df['gender'][item]:
        val11.append(new_df['percentage'][item])
    elif '25-34' in new_df['age'][item] and 'male' in new_df['gender'][item]:
        val12.append(new_df['percentage'][item])

unknown = pd.DataFrame({'25-34_unknown':val10})
female = pd.DataFrame({'25-34_female':val11})
male = pd.DataFrame({'25-34_male':val12})

In [33]:
out3 = pd.concat([out2, unknown, female, male], axis = 1)
out3

,35-44_unknown,35-44_female,35-44_male,65+_unknown,65+_female,65+_male,18-24_unknown,18-24_female,18-24_male,25-34_unknown,25-34_female,25-34_male
0,0.001653,0.108628,0.108628,0.000802,0.045846,0.024702,0.001453,0.064435,0.052009,0.003758,0.201373,0.169706
1,0.002191,0.197589,0.197589,0.000974,0.099343,0.041028,0.003117,0.005722,0.006696,0.000243,0.028001,0.030558
2,0.001645,0.106398,0.106398,0.000866,0.03056,0.017834,0.000678,0.101983,0.05558,0.006493,0.263267,0.177128
3,0.003457,0.278272,0.278272,0.001481,0.084938,0.033333,0.00041,0.006173,0.007407,0.001235,0.02321,0.022716
4,0.002714,0.101764,0.101764,0.027027,0.014247,0.025102,0.000137,0.06038,0.101764,0.002035,0.126187,0.221167
...,...,...,...,...,...,...,...,...,...,...,...,...
1215,NaN,NaN,NaN,NaN,0.263636,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,NaN,NaN,NaN,NaN,0.261261,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,NaN,NaN,NaN,NaN,0.281321,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1218,NaN,NaN,NaN,NaN,0.156894,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
val13=[]
val14=[]
val15=[]
for item in new_df.index:
    if '45-54' in new_df['age'][item] and 'unknown' in new_df['gender'][item]:
        val13.append(new_df['percentage'][item])
    elif '45-54' in new_df['age'][item] and 'female' in new_df['gender'][item]:
        val14.append(new_df['percentage'][item])
    elif '45-54' in new_df['age'][item] and 'male' in new_df['gender'][item]:
        val15.append(new_df['percentage'][item])

unknown = pd.DataFrame({'45-54_unknown':val13})
female = pd.DataFrame({'45-54_female':val14})
male = pd.DataFrame({'45-54_male':val15})

In [35]:
out4 = pd.concat([out3, unknown, female, male], axis = 1)
out4

,35-44_unknown,35-44_female,35-44_male,65+_unknown,65+_female,65+_male,18-24_unknown,18-24_female,18-24_male,25-34_unknown,25-34_female,25-34_male,45-54_unknown,45-54_female,45-54_male
0,0.001653,0.108628,0.108628,0.000802,0.045846,0.024702,0.001453,0.064435,0.052009,0.003758,0.201373,0.169706,0.000551,0.069145,0.058523
1,0.002191,0.197589,0.197589,0.000974,0.099343,0.041028,0.003117,0.005722,0.006696,0.000243,0.028001,0.030558,0.001583,0.122717,0.104456
2,0.001645,0.106398,0.106398,0.000866,0.03056,0.017834,0.000678,0.101983,0.05558,0.006493,0.263267,0.177128,0.000693,0.047875,0.041382
3,0.003457,0.278272,0.278272,0.001481,0.084938,0.033333,0.00041,0.006173,0.007407,0.001235,0.02321,0.022716,0.002469,0.111358,0.091852
4,0.002714,0.101764,0.101764,0.027027,0.014247,0.025102,0.000137,0.06038,0.101764,0.002035,0.126187,0.221167,0.001357,0.050204,0.079376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,NaN,NaN,NaN,NaN,0.263636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,NaN,NaN,NaN,NaN,0.261261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,NaN,NaN,NaN,NaN,0.281321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1218,NaN,NaN,NaN,NaN,0.156894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
val16=[]
val17=[]
val18=[]
for item in new_df.index:
    if '55-64' in new_df['age'][item] and 'unknown' in new_df['gender'][item]:
        val16.append(new_df['percentage'][item])
    elif '55-64' in new_df['age'][item] and 'female' in new_df['gender'][item]:
        val17.append(new_df['percentage'][item])
    elif '55-64' in new_df['age'][item] and 'male' in new_df['gender'][item]:
        val18.append(new_df['percentage'][item])

unknown = pd.DataFrame({'55-64_unknown':val16})
female = pd.DataFrame({'55-64_female':val17})
male = pd.DataFrame({'55-64_male':val18})

In [37]:
out5 = pd.concat([out4, unknown, female, male], axis = 1)
out5

,35-44_unknown,35-44_female,35-44_male,65+_unknown,65+_female,65+_male,18-24_unknown,18-24_female,18-24_male,25-34_unknown,25-34_female,25-34_male,45-54_unknown,45-54_female,45-54_male,55-64_unknown,55-64_female,55-64_male
0,0.001653,0.108628,0.108628,0.000802,0.045846,0.024702,0.001453,0.064435,0.052009,0.003758,0.201373,0.169706,0.000551,0.069145,0.058523,0.000501,0.058573,0.039934
1,0.002191,0.197589,0.197589,0.000974,0.099343,0.041028,0.003117,0.005722,0.006696,0.000243,0.028001,0.030558,0.001583,0.122717,0.104456,0.00073,0.115291,0.065376
2,0.001645,0.106398,0.106398,0.000866,0.03056,0.017834,0.000678,0.101983,0.05558,0.006493,0.263267,0.177128,0.000693,0.047875,0.041382,0.000519,0.034802,0.022769
3,0.003457,0.278272,0.278272,0.001481,0.084938,0.033333,0.00041,0.006173,0.007407,0.001235,0.02321,0.022716,0.002469,0.111358,0.091852,0.001235,0.080988,0.04963
4,0.002714,0.101764,0.101764,0.027027,0.014247,0.025102,0.000137,0.06038,0.101764,0.002035,0.126187,0.221167,0.001357,0.050204,0.079376,0.00041,0.024423,0.032564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.157576,NaN
1252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.162162,NaN
1253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.155849,NaN
1254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.089487,NaN


In [38]:
val19=[]
val20=[]
val21=[]
for item in new_df.index:
    if '13-17' in new_df['age'][item] and 'unknown' in new_df['gender'][item]:
        val19.append(new_df['percentage'][item])
    elif '13-17' in new_df['age'][item] and 'female' in new_df['gender'][item]:
        val20.append(new_df['percentage'][item])
    elif '13-17' in new_df['age'][item] and 'male' in new_df['gender'][item]:
        val21.append(new_df['percentage'][item])

unknown = pd.DataFrame({'13-17_unknown':val19})
female = pd.DataFrame({'13-17_female':val20})
male = pd.DataFrame({'13-17_male':val21})

In [39]:
out6 = pd.concat([out5, unknown, female, male], axis = 1)
out6

,35-44_unknown,35-44_female,35-44_male,65+_unknown,65+_female,65+_male,18-24_unknown,18-24_female,18-24_male,25-34_unknown,...,25-34_male,45-54_unknown,45-54_female,45-54_male,55-64_unknown,55-64_female,55-64_male,13-17_unknown,13-17_female,13-17_male
0,0.001653,0.108628,0.108628,0.000802,0.045846,0.024702,0.001453,0.064435,0.052009,0.003758,...,0.169706,0.000551,0.069145,0.058523,0.000501,0.058573,0.039934,NaN,NaN,NaN
1,0.002191,0.197589,0.197589,0.000974,0.099343,0.041028,0.003117,0.005722,0.006696,0.000243,...,0.030558,0.001583,0.122717,0.104456,0.00073,0.115291,0.065376,NaN,NaN,NaN
2,0.001645,0.106398,0.106398,0.000866,0.03056,0.017834,0.000678,0.101983,0.05558,0.006493,...,0.177128,0.000693,0.047875,0.041382,0.000519,0.034802,0.022769,NaN,NaN,NaN
3,0.003457,0.278272,0.278272,0.001481,0.084938,0.033333,0.00041,0.006173,0.007407,0.001235,...,0.022716,0.002469,0.111358,0.091852,0.001235,0.080988,0.04963,NaN,NaN,NaN
4,0.002714,0.101764,0.101764,0.027027,0.014247,0.025102,0.000137,0.06038,0.101764,0.002035,...,0.221167,0.001357,0.050204,0.079376,0.00041,0.024423,0.032564,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.157576,NaN,NaN,NaN,NaN
1252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.162162,NaN,NaN,NaN,NaN
1253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.155849,NaN,NaN,NaN,NaN
1254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.089487,NaN,NaN,NaN,NaN


In [40]:
real_df = pd.concat([df7, out6], axis = 1)
real_df.drop('demographic_distribution', axis = 1, inplace=True)
real_df

,Alabama_region_distribution,Alaska_region_distribution,Arizona_region_distribution,Arkansas_region_distribution,California_region_distribution,Colorado_region_distribution,Connecticut_region_distribution,Delaware_region_distribution,Florida_region_distribution,Georgia_region_distribution,...,25-34_male,45-54_unknown,45-54_female,45-54_male,55-64_unknown,55-64_female,55-64_male,13-17_unknown,13-17_female,13-17_male
0,0,0,0,0,0,0,0,0,0,0,...,0.169706,0.000551,0.069145,0.058523,0.000501,0.058573,0.039934,NaN,NaN,NaN
1,0,0,0,0,0,0,0,0,0,0,...,0.030558,0.001583,0.122717,0.104456,0.00073,0.115291,0.065376,NaN,NaN,NaN
2,0,0,0,0,0,0,0,0,0,0,...,0.177128,0.000693,0.047875,0.041382,0.000519,0.034802,0.022769,NaN,NaN,NaN
3,0,0,0,0,0,0,0,0,0,0,...,0.022716,0.002469,0.111358,0.091852,0.001235,0.080988,0.04963,NaN,NaN,NaN
4,0,0,0,0,0,0,0,0,0,0,...,0.221167,0.001357,0.050204,0.079376,0.00041,0.024423,0.032564,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16042,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16043,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16044,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16045,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
real_df["ad_creation_time"] = pd.to_datetime(real_df["ad_creation_time"])
real_df["ad_creation_time"]

0       2020-10-24
1       2020-10-24
2       2020-10-24
3       2020-10-24
4       2020-10-24
           ...    
16042          NaT
16043          NaT
16044          NaT
16045          NaT
16046          NaT
Name: ad_creation_time, Length: 16047, dtype: datetime64[ns]

In [42]:
mask = (real_df["ad_creation_time"] > '2020-08-01') & (real_df["ad_creation_time"] <= '2020-11-03')

In [43]:
real_df = real_df.loc[mask]
#real_df["ad_creation_time"]
distinct_date = pd.unique(real_df["ad_creation_time"])
print(distinct_date)


['2020-10-24T00:00:00.000000000' '2020-10-23T00:00:00.000000000'
 '2020-10-22T00:00:00.000000000' '2020-10-21T00:00:00.000000000'
 '2020-10-20T00:00:00.000000000' '2020-10-19T00:00:00.000000000'
 '2020-10-15T00:00:00.000000000' '2020-10-13T00:00:00.000000000'
 '2020-10-11T00:00:00.000000000' '2020-10-07T00:00:00.000000000'
 '2020-10-02T00:00:00.000000000' '2020-10-01T00:00:00.000000000'
 '2020-09-29T00:00:00.000000000' '2020-09-28T00:00:00.000000000'
 '2020-09-26T00:00:00.000000000' '2020-09-25T00:00:00.000000000'
 '2020-09-24T00:00:00.000000000' '2020-09-21T00:00:00.000000000'
 '2020-09-19T00:00:00.000000000' '2020-09-16T00:00:00.000000000'
 '2020-09-15T00:00:00.000000000' '2020-09-14T00:00:00.000000000'
 '2020-09-13T00:00:00.000000000' '2020-09-10T00:00:00.000000000'
 '2020-09-07T00:00:00.000000000' '2020-09-02T00:00:00.000000000'
 '2020-08-31T00:00:00.000000000' '2020-08-27T00:00:00.000000000'
 '2020-08-23T00:00:00.000000000' '2020-08-18T00:00:00.000000000'
 '2020-08-13T00:00:00.000

In [44]:
real_df = real_df.fillna(value=0, axis = 1)

## DON'T FORGET TO CHANGE THE PATH NAME AND THE FILE NAME.

In [46]:
#real_df.to_csv(r'C:\Users\Aney\OneDrive - Central Michigan University\PlanB\candidateCleanUp\TinaSmith.csv', index = False)